In [17]:
using System.IO;

var input = File.ReadAllLines("input.txt");

var q = from y in Enumerable.Range(0, input.Length)
        from x in Enumerable.Range(0, input[y].Length)
        select (x, y, c:input[y][x]);

var grid = q.ToDictionary(x => new Coordinate(x.x,x.y), x => x.c);
var locations  = grid
    .Where(kv => char.IsDigit(kv.Value)).Select(kv => kv.Key)
    .ToList();
var start = locations.Single(c => grid[c] == '0');
var maxX = grid.Max(kv => kv.Key.x);
var maxY = grid.Max(kv => kv.Key.y);

var distances = (
    from src in locations
    from dst in locations
    where src != dst
    let distance = Distance(src, dst)
    select (src, dst, distance)
    ).ToDictionary(x => (x.src,x.dst), x => x.distance);

var minimum = int.MaxValue;
foreach (var path in GetPermutations(locations.Except(new[]{start}), locations.Count - 1))
{
    var distance = 0;
    foreach (var (src, dst) in Windowed2(new[]{start}.Concat(path)))
    {
        distance += distances[(src, dst)];
    }
    minimum = Math.Min(distance, minimum);
}

minimum.Display();



int Distance(Coordinate from, Coordinate to)
{
    var queue = new Queue<(Coordinate to, int distance)>();
    queue.Enqueue((from, distance: 0));
    var visited = new HashSet<Coordinate> { from };
    while (queue.Any())
    {
        var (current, distance) = queue.Dequeue();
        if (current == to)
        {
            return distance;
        }
        foreach (var n in current.Neighbours(maxX, maxY))
        {
            if (!visited.Contains(n) && grid[n] != '#')
            {
                queue.Enqueue((n, distance + 1));
                visited.Add(n);
            }
        }
    }
    return -1;
}
IEnumerable<IEnumerable<T>> GetPermutations<T>(IEnumerable<T> list, int length) => length == 1
        ? list.Select(t => new[]{t})
        : from t in GetPermutations(list, length - 1)
          from e in list.Except(t)
          select t.Concat(new[]{e});

IEnumerable<(T a, T b)> Windowed2<T>(IEnumerable<T> list)
{
    var enumerator = list.GetEnumerator();
    if (!enumerator.MoveNext()) yield break;
    var a = enumerator.Current;
    while (true)
    {
        if (!enumerator.MoveNext()) yield break;
        var b = enumerator.Current;
        yield return (a, b);
        a = b;
    }
}

readonly record struct Coordinate(int x, int y)
{
    public IEnumerable<Coordinate> Neighbours(int maxX, int maxY)
    {
        if (x > 0) yield return this with { x = x - 1 };
        if (y > 0) yield return this with { y = y - 1 };
        if (x < maxX) yield return this with { x = x + 1 };
        if (y < maxY) yield return this with { y = y + 1 };
    }
}

502